# IBM QPU Usage Example

This notebook demonstrates how to use the `IBMQuantumRunner` class to run quantum circuits on IBM quantum computers.

## Setup and Import

First, let's import the necessary modules and create a simple quantum circuit.

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from quantum_studies.ibm_qpus import IBMQuantumRunner, run_on_ibm_qpu, list_available_backends
import os

# Replace with your actual IBM Quantum API key
# You can also set it as an environment variable: export IBM_QUANTUM_TOKEN=your_token
API_KEY = "your_api_key_here"  # or use os.getenv('IBM_QUANTUM_TOKEN')

## Method 1: Using the IBMQuantumRunner Class (Recommended)

This method gives you full control over the quantum backend operations.

In [ ]:
# Initialize the IBM Quantum Runner
runner = IBMQuantumRunner(api_key=API_KEY, channel="ibm_quantum_platform")

# List available backends
backends = runner.list_backends(min_qubits=2)

In [ ]:
# Create a Bell state circuit
qc = QuantumCircuit(2, 2)
qc.h(0)          # Apply Hadamard gate to qubit 0
qc.cx(0, 1)      # Apply CNOT gate from qubit 0 to qubit 1
qc.measure_all() # Measure all qubits

print("Bell State Circuit:")
print(qc.draw())

In [ ]:
# Select a backend (choose one from the list above)
backend_name = 'ibm_brisbane'  # Replace with an available backend
runner.select_backend(backend_name)

# Get backend information
backend_info = runner.get_backend_info()
print("Backend Information:")
for key, value in backend_info.items():
    print(f"{key}: {value}")

In [ ]:
# Run the circuit on the quantum computer
counts, result = runner.run_circuit(
    circuit=qc,
    shots=1024,
    optimization_level=1,
    plot_results=True,
    title="Bell State on IBM Quantum Computer"
)

# Calculate Bell state fidelity
fidelity = runner.calculate_bell_state_fidelity(counts)

## Method 2: Using Convenience Functions

For quick one-off executions, you can use the convenience functions.

In [ ]:
# List available backends quickly
available_backends = list_available_backends(API_KEY, min_qubits=2)

In [ ]:
# Run a circuit quickly
counts, result = run_on_ibm_qpu(
    circuit=qc,
    api_key=API_KEY,
    backend_name='ibm_brisbane',  # Replace with available backend
    shots=512
)

print("Quick execution results:", counts)

## Example: Running Multiple Circuits

Here's how you can run multiple different circuits using the same runner instance.

In [ ]:
# Create different quantum circuits

# 1. Simple superposition
superposition_circuit = QuantumCircuit(1, 1)
superposition_circuit.h(0)
superposition_circuit.measure(0, 0)

# 2. GHZ state (3-qubit entanglement)
ghz_circuit = QuantumCircuit(3, 3)
ghz_circuit.h(0)
ghz_circuit.cx(0, 1)
ghz_circuit.cx(1, 2)
ghz_circuit.measure_all()

print("Superposition Circuit:")
print(superposition_circuit.draw())
print("\nGHZ Circuit:")
print(ghz_circuit.draw())

In [ ]:
# Run superposition circuit
super_counts, super_result = runner.run_circuit(
    superposition_circuit,
    shots=1024,
    title="Superposition State"
)

In [ ]:
# Run GHZ circuit (if backend has enough qubits)
try:
    ghz_counts, ghz_result = runner.run_circuit(
        ghz_circuit,
        shots=1024,
        title="GHZ State"
    )
except Exception as e:
    print(f"Could not run GHZ circuit: {e}")
    print("Try selecting a backend with at least 3 qubits")

## Usage Tips

1. **API Key Management**: Store your API key as an environment variable for security:
   ```bash
   export IBM_QUANTUM_TOKEN=your_actual_token
   ```
   Then use: `API_KEY = os.getenv('IBM_QUANTUM_TOKEN')`

2. **Backend Selection**: Check the queue length and choose backends with shorter queues for faster execution.

3. **Circuit Optimization**: Use higher optimization levels (2-3) for better performance on real hardware.

4. **Error Handling**: The module includes comprehensive error handling and troubleshooting information.

5. **Reusability**: Create one runner instance and reuse it for multiple circuits to avoid repeated authentication.